# LINHA <> - Caderno de Operação

## Caderno de Inicialização e Calibração inicial da Linha de Luz

### Técnico responsável:

Texto inicial










## Notebook Initialization

The next cell loads the <b><i>util.ipynb</i></b> notebook, which provides functions and classes to connect the IJupyter and EPICS interfaces.

<sup><sup>Any bug or unexpected behavior from any of these functions and classes, please contact SOL.</sup></sup>

In [1]:
import io, os, sys, types
from IPython import get_ipython
import nbformat
from IPython.core.interactiveshell import InteractiveShell


def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, nbformat.NO_CONVERT)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [2]:
from util import *

importing Jupyter notebook from util.ipynb


## Configuration Cell

Check the boxes to set some notebook's configuration

In [11]:
checkbox_logprint_in_cell = widgets.Checkbox(
    value=False,
    description="Print log in Notebook's cells",
    disabled=False,
    style={'description_width': 'initial'},
)

config = {"log_cell": checkbox_logprint_in_cell}

output = widgets.Output()

@output.capture()
def change_checkbox_logprint_in_cell(change):
    logprint("Changed Logprint Checkbox to: " + str(change.new))
    
checkbox_logprint_in_cell.observe(change_checkbox_logprint_in_cell, 'value')

box = widgets.VBox([checkbox_logprint_in_cell, output])

display(box)



## Motor Calibration

### Write in the Text Box, separated by spaces, every motor name that should be initialized.

###### PS: Do not use any other character separation other than spaces!

In [ ]:
config

In [ ]:
start_motor_button_box = StartMotorsButton(config)
start_motor_button_box.display_start_button()

### Write in the "Number Text Box" the absolute target value desired for the motor 

In [ ]:
start_motor_button_box.display_motors_targ_buttons()

### Write in the Text Box, separated by spaces, every motor PV name that should be monitored.

###### PS: Do not use any other character separation other than spaces!

In [ ]:
motor_monitor = MotorsMonitor(config)
motor_monitor.display_monitor_motors()

In [ ]:
# from py4syn.utils.scan import scan
# scan(cnew_motor, 10, 5, 10, 0.1)

In [ ]:
pv_monitor = PVMonitor(config)
pv_monitor.display_monitor_pvs()

## Call Scan

In [ ]:
# import matplotlib
# matplotlib.get_backend()

In [ ]:
%pylab

%run untitled.py

In [ ]:
import matplotlib.pyplot as plt
import multiprocessing
from queue import Empty
from haha import call

# queue = None

%matplotlib

%tb

def poll_draw(queue, fig):
    def call_back(arg=None):
        try:
            while 1:
                try:
                    command = queue.get_nowait()
                except Empty:
                    break

                cmd = command['cmd']
                try:
                    idx = command['idx']
                except:
                    idx = -1

                if(cmd == "plot"):
                    x = np.linspace(0, 6*np.pi, 100)
                    y = np.sin(x)

                    ax = fig.add_subplot(111)
                    line1, = ax.plot(x, y, 'r-') 
                else:
                    pass # not implemented

            self.fig.canvas.draw()
            self.fig.canvas.flush_events()
        except Exception as e:
            pass
        return True
    return call_back

if __name__ == '__main__':
    
    
#     __spec__ = None
    fig = plt.figure()
    print("he")
    title = 'ha'
    plotsCount = 0
    ctx = multiprocessing.get_context('fork')  # @UndefinedVariable
    plot_queue =  ctx.Queue()
    plot_process = ctx.Process(target=call, args=(plot_queue, title, fig))
    plot_process.daemon = True
    plot_process.start()
    
    plot_process.join()

In [ ]:
matplotlib.rcsetup.all_backends

In [ ]:
# %matplotlib

import matplotlib

matplotlib.use("TkAgg", force=True)

import matplotlib.pyplot as plt

import py4syn.utils.plotter
py4syn.utils.plotter.main()

In [ ]:
main_module = sys.modules['__main__']
main_mod_name = getattr(main_module.__spec__, "name", None)
main_mod_name

In [ ]:
%matplotlib

scan -c default --start 0 --end 1 --step-or-point 0.1 --time 0.1 --motor solm1

In [ ]:
%matplotlib
import sys

if __name__ == '__main__':
#     subprocess.Popen(["/home/gabriel.andrade/work/scan-utils/scan", "-c", "default", "--start", "0", "--end", "1", "--step-or-point", "0.1", "--time", "0.1", "--motor", "solm1"])
    
    from scan_utils.scan import runScan
    import pylab
    
    sys.argv = ['-f'] + ["-c", "default", "--start", "0", "--end", "1", "--step-or-point", "0.1", "--time", "0.1", "--motor", "solm1"]
    %tb
    runScan()

#     pylab.show()

In [ ]:
%%script bash --bg --out output
echo "ha"
cd /home/gabriel.andrade/work/scan-gui/
./scan_gui

In [ ]:
!cd ~/work/scan-gui/ && ./scan_gui

In [ ]:
scan_but = ScanButton(config)
scan_but.display_scan()

In [ ]:
ha = 2

## Plotting data

### Plotly

Plotly is a great library for plotting data in a Jupyter Notebook.

Informations and examples of how to use this library can be found in https://plot.ly/python/


#### Simple scatter plot example:

In [ ]:
# Enable offline plots from plotly
py.init_notebook_mode(connected=True)

# Read sample csv file with pandas library (CSV example obatained from https://catalog.data.gov/dataset/age-adjusted-death-rates-for-the-top-10-leading-causes-of-death-united-states-2013/resource/0e603f1d-31bf-4809-8f10-a994b305b379)
df = pd.read_csv('sample-data.csv')

# Let's plot the number of total deaths for every USA state
data = df.groupby('State').sum().reset_index()

# Looking at our data, there is a United States row that includes the sum of all states, we should remove this row
for i in range(len(data.index) - 1):
    if data.iloc[i]['State'] == 'United States':
        data = data.drop(i, axis=0)

# Create a Scatter graph
trace1 = go.Scatter(
                    x=data['State'], y=data['Deaths'], # Data
                    mode='lines+markers', name='logx' # Additional options
                   )

# Create a Layout for the graph
layout = go.Layout(title='Number of deaths for every USA State',
                   plot_bgcolor='rgb(230, 230, 230)')

# Create a figure to be plotted
fig = go.FigureWidget(data=[trace1], layout=layout)

# Plot figure in the notebook
df.columns

In [ ]:
x = np.arange(1000) * 1/2
y = x * 2 + 9

s = pd.DataFrame({'x':x, 'y':y})
s

In [ ]:
default_name = 'scans/test_0012'

df = pd.read_csv(default_name, sep=' ', comment='#', header=None)

labels = []
with open(default_name) as file:
    for i, line in enumerate(file):
        if i == 7:
            labels = line.split(' ')[1:]

labels = list(filter(lambda x: x != '', labels))

filtered_label = []

for item in labels:
    filtered_label.append(item.rstrip('\n'))
    print(item)

df.columns = pd.Index(filtered_label, dtype='object')
df[df.columns[2]].values

In [ ]:
import json

j = json.load(open("./save"))
len(j['listMotors']['value'])

In [ ]:
trace1 = go.Scatter(
                    x=df.index.values, y=df['ringcurrent'], # Data
                    mode='lines+markers', name='line' # Additional options
                   )

# Create a Layout for the graph
layout = go.Layout(title='Number of deaths for every USA State',
                   plot_bgcolor='rgb(230, 230, 230)')

# Create a figure to be plotted
fig = go.FigureWidget(data=[trace1], layout=layout)

In [20]:
import threading
from IPython.display import display
import ipywidgets as widgets
import time
import pandas

import json

j = json.load(open("./save"))
j['listMotors']['value']

progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

trace1 = go.Scatter(
                    x=[], y=[], # Data
                    mode='lines+markers', name='line1' # Additional options
                   )

trace2 = go.Scatter(
                    x=[], y=[], # Data
                    mode='lines+markers', name='line2' # Additional options
                   )

# Create a Layout for the graph
layout = go.Layout(title='Scan',
                   plot_bgcolor='rgb(230, 230, 230)')

# Create a figure to be plotted
fig = go.FigureWidget(data=[trace1, trace2], layout=layout)

def update_pd(default_name, label):
    try:
        df = pd.read_csv(default_name, sep=' ', comment='#', header=None)
    except Exception as e:
        return pd.DataFrame(), label

    filtered_label = label
    if not label:
        labels = []
        with open(default_name) as file:
            for i, line in enumerate(file):
                if i == 8:
                    labels = line.split(' ')[1:]

        labels = list(filter(lambda x: x != '', labels))

        for item in labels:
            filtered_label.append(item.rstrip('\n'))
        
        label = filtered_label

    df.columns = pd.Index(filtered_label, dtype='object')
    return df, label

def work(figure, default_name, js):
    total = 200
    label = []
    
    for i in range(total):
        df, label = update_pd(default_name, label)
#         print(df)
        if df.empty:
            continue

        number_motors = len(js['listMotors']['value'])

        for i in range(len(df.columns) - number_motors): 
#             print(i)
    #         print(len(figure['data']))
            figure['data'][i]['x'] = df.index.values
            figure['data'][i]['y'] = df[df.columns[number_motors + i]].values
        
        time.sleep(1)
%tb
        
thread = threading.Thread(target=work, args=(fig, "/home/gabriel.andrade/work/jupy4syn/scans/plot_0025", j))
display(fig)
thread.start()

NameError: name 'config' is not defined

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'line1',
              'type': 'sc…

In [ ]:
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

def work(progress):
    total = 100
    for i in range(total):
        time.sleep(0.2)
        progress.value = float(i+1)/total

thread = threading.Thread(target=work, args=(progress,))
display(progress)
thread.start()

In [ ]:
fig['data'][1]['y']

In [ ]:
%tb
work(fig, "/home/gabriel.andrade/work/jupy4syn/scans/plot_0006", j)

In [ ]:
fig['data'][0]['y']

In [7]:
import json
import os

class MonitorScanSave(widgets.Button):
    
    def __init__(self, config, *args, **kwargs):
        widgets.Button.__init__(self, *args, **kwargs)
        
        # Config
        self.config = config
        
        # class Button values for MonitorScanSave
        self.description='Start Scanning Plot'
        self.disabled=False
        self.button_style='success'
        self.tooltip='Click me'
        self.icon=''
        self.layout = widgets.Layout(width='300px')
        
        # Scan save file and directory
        self.scan_save_dir = '/home/gabriel.andrade/work/.tmp/solshua/'
        self.scan_save_file = 'haha.save'

        self.scan_path = Path(self.scan_save_dir + self.scan_save_file)
        
        # Logging
        self.output = widgets.Output()
        
        # Threading
        self.monitor = False
        self.thread = threading.Thread(target=self.monitor_save_file)
        
        # Set callback function for click event
        self.on_click(self._start_button)
        
        # Widgets displays
        self.start_button = widgets.VBox([self])
        
        
    @staticmethod
    def _start_button(b):
        # Clear previous logs outputs
        b.output.clear_output()
        
        # with statement to output logs in stdou (if this option is enabled)
        with b.output:
            if b.monitor:
                # Change button to a "clicked status"
                b.disabled = True
                b.button_style = ''
                b.description='Stopping...'
                
                # We should sleep for some time to give some responsiveness to the user
                time.sleep(0.5)

                # Stop thread to monitor the save file
                try:
                    logprint("Stopping thread", config=b.config)
                    b.thread.join()
                except Exception as e:
                    # If any error occurs, log that but dont stop code exection
                    logprint("Error in stopping thread", "[ERROR]", config=b.config)
                    logprint(str(e), "[ERROR]", config=b.config)

                # Change button layout monitoring
                b.disabled = False
                b.button_style = 'success'
                b.description='Start Scanning Plot'
            else:
                # Change button to a "clicked status"
                b.disabled = True
                b.button_style = ''
                b.description='Starting...'
                
                # We should sleep for some time to give some responsiveness to the user
                time.sleep(0.5)

                # Start thread to monitor the save file
                try:
                    logprint("Starting thread", config=b.config)
                    b.thread.start()
                except Exception as e:
                    # If any error occurs, log that but dont stop code exection
                    logprint("Error in starting thread", "[ERROR]", config=b.config)
                    logprint(str(e), "[ERROR]", config=b.config)

                # Change button layout monitoring
                b.disabled = False
                b.button_style = 'danger'
                b.description='Stop Scanning Plot'
            
            # Change button monitor status
            b.monitor = not b.monitor
    
    def monitor_save_file(self):
        with self.output:
            while self.monitor:
                logprint("haha", config=self.config)
                if self.scan_path.is_file():
                    logprint("ha", config=self.config)
                    save_file = []
                    with open(str(self.scan_path)) as file:
                        try: 
                             save_file = json.load(file)
                        except ValueError: 
                             pass
                    logprint("ha1", config=self.config)
                    os.remove(str(self.scan_path))
                    logprint("ha2", config=self.config)

                # Call graph
                time.sleep(0.5)
    
    def display_start_button(self):
        display(self.start_button, self.output)


In [21]:
scan_save = MonitorScanSave(config)
scan_save.display_start_button()

Output()

In [ ]:
scan_save_dir = '/home/gabriel.andrade/work/.tmp/solshua/'
scan_save_file = 'haha.save'

scan_path = Path(scan_save_dir + scan_save_file)
scan_path.is_file()

In [ ]:
scan_save.monitor